# F4 Pre Trained Autoencoders

From:

- https://github.com/CompVis/latent-diffusion#pretrained-autoencoding-models




### KL Autoencoder

In [9]:
import os
import wget
import zipfile

def download_pre_trained_ae(url, output_dir):
    if os.path.exists(output_dir):
        print("Used cache")
    else:
        filename = wget.download(url)

        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(output_dir)

        if os.path.exists(filename):
            os.remove(filename)

In [10]:
output_dir = './pretrained_models/kl-f4'
url = "https://ommer-lab.com/files/latent-diffusion/kl-f4.zip"

download_pre_trained_ae(url, output_dir)

Used cache


In [3]:
%pip uninstall taming-transformers

^C
Note: you may need to restart the kernel to use updated packages.


In [7]:
!git mv taming latent_diffusion_compvis/taming

The file will have its original line endings in your working directory


In [13]:
!git log --branches --not --remotes

In [3]:
!git reset --soft HEAD~1 3e9a62573568dd4d00ca61c5c3da76c2271d2488

fatal: Cannot do soft reset with paths.


In [10]:
!git reset --soft 8e4cee53fcb8d01ec58f85638a2097097399782a

In [21]:
!git filter-branch --index-filter "git rm -rf --cached --ignore-unmatch autoencoder/pretrained_models/kl-f4/model.ckpt" HEAD

Cannot rewrite branches: You have unstaged changes.



	 rewrites.  Hit Ctrl-C before proceeding to abort, then use an
	 alternative filtering tool such as 'git filter-repo'
	 (https://github.com/newren/git-filter-repo/) instead.  See the
	 filter-branch manual page for more details; to squelch this warning,
	 set FILTER_BRANCH_SQUELCH_WARNING=1.
Proceeding with filter-branch...



In [ ]:
!git filter-branch --index-filter "git rm -rf --cached --ignore-unmatch autoencoder/kl-f4.zip" HEAD

In [22]:
!git filter-branch --force --index-filter 'git rm --cached --ignore-unmatch autoencoder/kl-f4.zip' --prune-empty --tag-name-filter cat -- --all

	 rewrites.  Hit Ctrl-C before proceeding to abort, then use an
	 alternative filtering tool such as 'git filter-repo'
	 (https://github.com/newren/git-filter-repo/) instead.  See the
	 filter-branch manual page for more details; to squelch this warning,
	 set FILTER_BRANCH_SQUELCH_WARNING=1.
Proceeding with filter-branch...



Cannot rewrite branches: You have unstaged changes.


In [23]:
!git filter-branch --force --index-filter 'git rm --cached --ignore-unmatch autoencoder/pretrained_models/kl-f4/model.ckpt' --prune-empty --tag-name-filter cat -- --all

	 rewrites.  Hit Ctrl-C before proceeding to abort, then use an
	 alternative filtering tool such as 'git filter-repo'
	 (https://github.com/newren/git-filter-repo/) instead.  See the
	 filter-branch manual page for more details; to squelch this warning,
	 set FILTER_BRANCH_SQUELCH_WARNING=1.
Proceeding with filter-branch...



Cannot rewrite branches: You have unstaged changes.


In [12]:
!git add .


In [18]:
!git reset HEAD autoencoder/pretrained_models/kl-f4/model.ckpt
!git reset HEAD autoencoder/kl-f4.zip

Unstaged changes after reset:
D	autoencoder/pretrained_models/kl-f4/model.ckpt
M	diffusion_f4_ae.ipynb
Unstaged changes after reset:
D	autoencoder/kl-f4.zip
D	autoencoder/pretrained_models/kl-f4/model.ckpt
M	diffusion_f4_ae.ipynb


In [19]:
!git rm --cached autoencoder/pretrained_models/kl-f4/model.ckpt
!git rm --cached autoencoder/kl-f4.zip

rm 'autoencoder/pretrained_models/kl-f4/model.ckpt'
rm 'autoencoder/kl-f4.zip'


In [11]:
!git commit --amend 

^C


In [ ]:
!git commit --amend 

In [4]:
from latent_diffusion_compvis.ldm.models.autoencoder import AutoencoderKL

ModuleNotFoundError: No module named 'ldm'

In [13]:
from taming.modules.vqvae.quantize import VectorQuantizer2

ImportError: cannot import name 'VectorQuantizer2' from 'taming.modules.vqvae.quantize' (c:\Users\willf\AppData\Local\Programs\Python\Python39\lib\site-packages\taming\modules\vqvae\quantize.py)

In [1]:
from taming.modules.vqvae.quantize import VectorQuantizer2

ImportError: cannot import name 'VectorQuantizer2' from 'taming.modules.vqvae.quantize' (c:\Users\willf\AppData\Local\Programs\Python\Python39\lib\site-packages\taming\modules\vqvae\quantize.py)